In [ ]:
!pip uninstall langchain
!pip install langchain
!pip install neo4j
!pip install bs4
!pip install llama-index
!pip uninstall -y trulens_eval
#!pip install trulens-eval==0.25.1
!pip install llmlingua
!pip install regex

In [ ]:
!pip uninstall -y trulens_eval
!pip install trulens-eval

In [1]:
import os
from bs4 import BeautifulSoup
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
import warnings
from langchain_community.document_loaders import DirectoryLoader
from llama_index.core import SimpleDirectoryReader, StorageContext
warnings.filterwarnings("ignore")
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
import numpy as np
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
from llmlingua import PromptCompressor
import re
from IPython.core.display import display, HTML

In [2]:
def word_wrap(string, n_chars=72):
    # Wrap a string at the next space after n_chars
    if len(string) < n_chars:
        return string
    else:
        return string[:n_chars].rsplit(' ', 1)[0] + '\n' + word_wrap(string[len(string[:n_chars].rsplit(' ', 1)[0])+1:], n_chars)

In [3]:
NEO4J_URI = 'bolt://' + os.getenv('NEO4J_HOST') + ':7687'
NEO4J_USERNAME = os.getenv('NEO4J_USER')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = 'neo4j' #os.getenv('NEO4J_DB')
print(NEO4J_URI)
print(NEO4J_DATABASE)

bolt://neo4j:7687
neo4j


In [4]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [5]:
cypher = """
  MATCH (n) 
  RETURN count(n)
  """
result = kg.query(cypher)
result

[{'count(n)': 114381}]

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    separators=["\n\n", "\n", ". ", " ", ""],
)

In [7]:
file_metadata = lambda x: {"filename": x}
Acts_documents = SimpleDirectoryReader("./XML",file_metadata=file_metadata).load_data()

In [8]:
act_data = Acts_documents[4].get_text()

In [9]:
def get_title(soup):
    title = soup.find_all("h2")
    title = title[0].get_text().strip()
    return title

In [10]:
def get_definitions(sections):
    for index, section in enumerate(sections):
        heading = section.find("h4")
        print(heading.get_text())
        if 'Definition' in heading.get_text():
            definition = section
            return definition

In [11]:
def get_preamble(soup):
    preamble = soup.find_all("div", class_='preamble')
    if preamble:
        print(preamble[0].get_text())

In [12]:
file = BeautifulSoup(act_data, "lxml")

In [13]:
print(file)

<?xml version="1.0" encoding="UTF-8"?><!-- This information is licensed under the Queen's Printer License of British Columbia found: www.bclaws.ca/standards/2014/QP-License_1.0.html. Please review the license terms carefully. --><html><body><act:act id="00_96006_01" xmlns:act="http://www.gov.bc.ca/2013/legislation/act" xmlns:bcl="http://www.gov.bc.ca/2013/bclegislation" xmlns:fun="http://www.bclaws.ca/xslt/functions" xmlns:in="http://www.qp.gov.bc.ca/2013/inline" xmlns:oasis="http://docs.oasis-open.org/ns/oasis-exchange/table" xmlns:reg="http://www.gov.bc.ca/2013/legislation/regulation" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemalocation="http://www.gov.bc.ca/2013/legislation/act http://standards.qp.gov.bc.ca/standards/2013/act_1.0.xsd" xsl-fo="https://styles.qp.gov.bc.ca/bclaws-xsl-fo/act.xsl">
<act:title>Adult Guardianship Act</act:title>
<act:chapter>6</act:chapter>
<act:yearenacted>1996</act:yearenacted>
<act:currency>

In [14]:
title = file.find_all('act:title')
print(title)

[<act:title>Adult Guardianship Act</act:title>]


In [15]:
definitions = file.find_all('bcl:definition')
print(definitions)

[<bcl:definition id="d2e41">
<bcl:text>
<in:term>abuse</in:term> means the deliberate mistreatment of an adult that causes the
            adult</bcl:text>
<bcl:paragraph id="d2e50">
<bcl:num>a</bcl:num>
<bcl:text>physical, mental or emotional harm, or</bcl:text>
</bcl:paragraph>
<bcl:paragraph id="d2e60">
<bcl:num>b</bcl:num>
<bcl:text>damage or loss in respect of the adult's financial affairs,</bcl:text>
</bcl:paragraph>
<bcl:text>and includes intimidation, humiliation, physical assault, sexual assault,
            overmedication, withholding needed medication, censoring mail, invasion or denial of
            privacy or denial of access to visitors;</bcl:text>
</bcl:definition>, <bcl:definition id="d2e72">
<bcl:text>
<in:term>adult</in:term> means anyone who has reached 19 years of age and, for all
            purposes incidental to an application under section 6 (2), includes a person who has reached 18
            years of age;</bcl:text>
</bcl:definition>, <bcl:definition id="d2e

In [16]:
num_sections = file.find_all('bcl:section')
print(num_sections)
print(len(num_sections)[0])

[<bcl:section id="d2e30">
<bcl:marginalnote>Definitions</bcl:marginalnote>
<bcl:num>1</bcl:num>
<bcl:text>In this Act:</bcl:text>
<bcl:definition id="d2e41">
<bcl:text>
<in:term>abuse</in:term> means the deliberate mistreatment of an adult that causes the
            adult</bcl:text>
<bcl:paragraph id="d2e50">
<bcl:num>a</bcl:num>
<bcl:text>physical, mental or emotional harm, or</bcl:text>
</bcl:paragraph>
<bcl:paragraph id="d2e60">
<bcl:num>b</bcl:num>
<bcl:text>damage or loss in respect of the adult's financial affairs,</bcl:text>
</bcl:paragraph>
<bcl:text>and includes intimidation, humiliation, physical assault, sexual assault,
            overmedication, withholding needed medication, censoring mail, invasion or denial of
            privacy or denial of access to visitors;</bcl:text>
</bcl:definition>
<bcl:definition id="d2e72">
<bcl:text>
<in:term>adult</in:term> means anyone who has reached 19 years of age and, for all
            purposes incidental to an application under sec

TypeError: 'int' object is not subscriptable

## Working with the 1st section

#### bcl - B.C Laws #####
bcl:num gives the number of sections, subsections and something more

bcl:section gives all the sections and subsections

bcl:marginalnote gices the section heading

if the section is a definition then each definition has a definition tag

In [24]:
section_1 = num_sections[2]
section_heading = section_1.find_all('bcl:marginalnote')
section_definitions = section_1.find_all('bcl:definition')
section_subsection = section_1.find_all('bcl:subsection')
print(section_1)
print(len(section_subsection))

<bcl:section id="d2e384">
<bcl:marginalnote>Presumption of capability</bcl:marginalnote>
<bcl:num>3</bcl:num>
<bcl:subsection id="d2e392">
<bcl:num>1</bcl:num>
<bcl:text>Until the contrary is demonstrated, every adult is presumed to be capable of
            making decisions about the adult's personal care, health care and financial
            affairs.</bcl:text>
</bcl:subsection>
<bcl:subsection id="d2e401">
<bcl:num>2</bcl:num>
<bcl:text>An adult's way of communicating with others is not grounds for deciding that he
            or she is incapable of making decisions about anything referred to in subsection
              (1).</bcl:text>
</bcl:subsection>
</bcl:section>
2


In [25]:
cache = []

In [26]:
#def link_references(str, index):
    

In [27]:
merge_chunk_node_query = """
MERGE(mergedChunk:UpdatedChunk {chunkId: $chunkParam.chunkId})
    ON CREATE SET 
        mergedChunk.chunkSeqId = $chunkParam.chunkSeqId, 
        mergedChunk.text = $chunkParam.text,
        mergedChunk.ActId = $chunkParam.ActId,
        mergedChunk.sectionId = $chunkParam.sectionId,
        mergedChunk.sectionName = $chunkParam.sectionName
RETURN mergedChunk
"""

In [28]:
match_chunk_node_query =  """
        MATCH (chunk:UpdatedChunk) WHERE
        chunk.chunkId = $chunkParam.chunkId
        AND chunk.chunkSeqId = $chunkParam.chunkSeqId
        AND chunk.ActId = $chunkParam.ActId
        AND chunk.sectionId = $chunkParam.sectionId
        AND chunk.sectionName = $chunkParam.sectionName
        AND chunk.text = $chunkParam.text
        RETURN chunk
        """

In [29]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [30]:
create_embeddings = """
        MATCH (chunk:UpdatedChunk) WHERE
        chunk.chunkId = $chunkParam.chunkId
        AND chunk.chunkSeqId = $chunkParam.chunkSeqId
        AND chunk.ActId = $chunkParam.ActId
        AND chunk.sectionId = $chunkParam.sectionId
        AND chunk.sectionName = $chunkParam.sectionName
        AND chunk.text = $chunkParam.text
        AND chunk.textEmbedding is NULL
        CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", $vector)
        RETURN chunk
    """


In [31]:
connect_chunks = """
      MATCH (chunk:UpdatedChunk), (f:UpdatedChunk)
      WHERE
        chunk.chunkId = $chunkParam.chunkId
        AND chunk.chunkSeqId = $chunkParam.chunkSeqId
        AND chunk.ActId = $chunkParam.ActId
        AND chunk.sectionId = $chunkParam.sectionId
        AND chunk.sectionName = $chunkParam.sectionName
        AND f.ActId = $chunkParam.ActId
        AND f.sectionId = $chunkParam.connectnedsectionId
        AND f.chunkSeqId = 0
      MERGE (chunk)-[newRelationship:REFERENCE]->(f)
      RETURN count(newRelationship)
    """

In [32]:
def create_chunk_relationship(act_info):
    cypher = """
  MATCH (from_same_section:UpdatedChunk)
  WHERE from_same_section.ActId = $ActParam['ActId']
  AND from_same_section.sectionName = $ActParam['sectionName']
  AND from_same_section.sectionId = $ActParam['sectionId']
  WITH from_same_section
    ORDER BY from_same_section.chunkSeqId ASC
  WITH collect(from_same_section) as section_chunk_list
    CALL apoc.nodes.link(
        section_chunk_list, 
        "NEXT", 
        {avoidDuplicates: true}
    )  // NEW!!!
  RETURN size(section_chunk_list)
"""
    kg.query(cypher, params={'ActParam': act_info})

In [33]:
## This function will search for any reference that has the word section or subsection followed by a number
def extract_references(str, index):
    references = re.findall(r"section (\d+)", str)
    #print(references)
    #if (len(references)):
        #link_references(str, index)
        #print(index)
        #cache[index].append(references)
print(cache)        

[]


In [39]:
def find_links(subsection):
    if not subsection:
        return
    if (subsection.find_all("bcl:link")):
        xml_link = subsection.find_all("bcl:link")[0]['xlink:href']
        display(HTML(f'<a href="{xml_link}">{subsection.find_all("bcl:link")[0].get_text()}</a>'))
    extract_references(subsection.get_text().replace("\n\n", "").replace("\r", ""), index)
    #print(subsection)
    #print(subsection.get_text().replace("\n", " ").replace("\r", " "))
    return subsection.get_text().replace("\n", " ").replace("\r", " ")

In [40]:
#get subsection
def subsection(section_definitions, index):
    string = ""
    for subsection_index, subsection in enumerate(section_definitions):
        #nested_section = subsection.find_all('bcl:num')
        string += "\n" + find_links(subsection)
    return string

In [41]:
def create_metadata(token_split_texts,title, section_heading, section_id):
    chunks_with_metadata = [] # use this to accumlate chunk records
    chunk_seq_id = 0
    for chunk in token_split_texts: # only take the first 20 chunks
        #form_id = file[file.rindex('/') + 1:file.rindex('.')] # extract form id from file name
        # finally, construct a record with metadata and the chunk text
        chunks_with_metadata.append({
            'text': chunk, 
            # metadata from looping...
            'chunkSeqId': chunk_seq_id,
            'chunkId': f'{title}-chunk-{section_heading}-{chunk_seq_id:04d}',
            'ActId': f'{title}',
            'sectionId': f'{section_id}',
            'sectionName':f'{section_heading}',
            # constructed metadata...
            # metadata from file...
        })
        chunk_seq_id += 1
    return chunks_with_metadata

In [42]:
def create_chunks(item_text, title, section_heading, section_id):
    item_text_chunks = text_splitter.split_text(item_text) # split the text into chunks
    token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=20, tokens_per_chunk=256)
    token_split_texts = []
    for text in item_text_chunks:
        token_split_texts += token_splitter.split_text(text)
    meta_data = create_metadata(token_split_texts,title, section_heading, section_id)    
    return meta_data

In [43]:
def search_create_reference(match):
    #print(match[0]['chunk'])
    text = match[0]['chunk']['text']
    references = re.findall(r"(?i)(?:section)\s+(\d+|\(\d+\))(?:\s*\([a-z]\))?", text)
    if (references):
        print(match[0]['chunk'])
        print(references)
        # create the edges
        print("Match found - creating references")
        chunk_seq_id = match[0]['chunk']['chunkSeqId']
        section_heading = match[0]['chunk']['sectionName']
        section_id = match[0]['chunk']['sectionId']
        chunk = {
            'text': text, 
            # metadata from looping...
            'chunkSeqId': match[0]['chunk']['chunkSeqId'],
            'chunkId': match[0]['chunk']['chunkId'],
            'ActId': match[0]['chunk']['ActId'],
            'sectionId': match[0]['chunk']['sectionId'],
            'sectionName': match[0]['chunk']['sectionName'],
            'connectnedsectionId': references[0]
        }
        result = kg.query(connect_chunks,
                params={
                    'chunkParam':chunk
                }
                )
        print(result)

In [44]:
def create_chunk_embeddings(tokens):
    for chunk in tokens:
        print(f"Creating `:Chunk` node for chunk ID {chunk['chunkSeqId']}")
        kg.query(merge_chunk_node_query, 
            params={
                'chunkParam': chunk
            })
        vector = embeddings.embed_query(chunk['text'])
        result = kg.query(create_embeddings, params={'chunkParam':chunk, 'vector':vector})
        if result:
            print("Embedding created")
        else:
            print(result)
            print("Embedding not created")
    create_chunk_relationship(tokens[0])       

In [45]:
def create_chunk_neo4j(tokens, search=False):
    for chunk in tokens:
        if search:
            vector = embeddings.embed_query(chunk['text'])
            result = kg.query(create_embeddings, params={'chunkParam':chunk, 'vector':vector})
            if result:
                print("Embedding creating")
            else:
                print(result)
                return
        else:
            print(f"Creating `:Chunk` node for chunk ID {chunk['chunkSeqId']}")
            kg.query(merge_chunk_node_query, 
                    params={
                        'chunkParam': chunk
                    })
    if (not search):
        create_chunk_relationship(tokens[0])
    #return match_found

In [46]:
def extract_data(file, index,  search=False):
    #get the ACT's title
    title = file.find('act:title')
    if title:
        title = title.get_text()
        print(title)
    else:
        return
    #get all the sections
    preamble = file.find_all('bcl:preamble')
    if (preamble):
        item_text = subsection(preamble, 0)
        token = create_chunks(item_text, title, 'preamble', 0)
    sections = file.find_all('bcl:section')
    #find the definition subsection
    for index, section in enumerate(sections):
        token = []
        section_heading = section.find('bcl:marginalnote')#[0].get_text()
        if (section_heading):
            section_heading = section_heading.get_text()
        else:
            section_heading = ""
        section_definitions = section.find_all('bcl:definition')
        if (len(section_definitions) < 1):
            #find the remaining subsection
            section_subsection = section.find_all('bcl:subsection')
            #find the section number
            section_number = section.find('bcl:num').get_text()
            if len(section_number):
                print("section number is:" + section_number)
            #print(section_subsection)
            if len(section_subsection):
                item_text = subsection(section_subsection, index+1)
                print(item_text)
                token = create_chunks(item_text, title, section_heading, section_number)
            else:
                item_text = find_links(section)
                token = create_chunks(item_text, title, section_heading, section_number)
        else:
            item_text = subsection(section_definitions, index+1)
            #print(item_text)
            token = create_chunks(item_text, title, section_heading, index+1)
        found = create_chunk_embeddings(token)

# This cell will create chuncks based on sections of the Acts

In [47]:
def check_act_exists(ActId):
    checkact = """
        MATCH (n:UpdatedChunk) where n.ActId=$ActId 
        return distinct n
    """
    return kg.query(checkact, params={'ActId': ActId})

In [58]:
corrupt = """
	MATCH (n:UpdatedChunk) where n.textEmbedding is NULL return distinct (n.ActId) as ActId
    """
corrupted_acts = kg.query(corrupt)
print(corrupted_acts)

[{'ActId': 'Motor Vehicle Act'}]


In [54]:
def delete_corrupted_act(ActId):
    delete_corrupt = """
        MATCH (n:UpdatedChunk) where n.ActId=$ActId 
        DETACH
        DELETE n
    """
    kg.query(delete_corrupt, params={'ActId': ActId})

In [59]:
def check_corrupted(title):
    found = False
    for corrupted in corrupted_acts:
        if title == corrupted['ActId']:
            found = True
            break
    return found

In [53]:
act = check_act_exists("Access to Abortion Services Act")
print(len(act))

26


In [60]:
corrupt_count = 0
for index, Acts in enumerate(Acts_documents):
    soup = BeautifulSoup(Acts.get_text(), 'xml')
    title = soup.find("act:title")
    if (title):
        title = title.get_text()
        if (check_corrupted(title)):
            print(title)
            print("Creating Chunks and Embeddings")
            delete_corrupted_act(title)
            extract_data(soup, index,  False)
            corrupt_count +=1
            #break
print(corrupt_count)            

Motor Vehicle Act
Creating Chunks and Embeddings
Motor Vehicle Act


Creating `:Chunk` node for chunk ID 0
Embedding created
Creating `:Chunk` node for chunk ID 1
Embedding created
Creating `:Chunk` node for chunk ID 2
Embedding created
Creating `:Chunk` node for chunk ID 3
Embedding created
Creating `:Chunk` node for chunk ID 4
Embedding created
Creating `:Chunk` node for chunk ID 5
Embedding created
Creating `:Chunk` node for chunk ID 6
Embedding created
Creating `:Chunk` node for chunk ID 7
Embedding created
Creating `:Chunk` node for chunk ID 8
Embedding created
Creating `:Chunk` node for chunk ID 9
Embedding created
Creating `:Chunk` node for chunk ID 10
Embedding created
Creating `:Chunk` node for chunk ID 11
Embedding created
Creating `:Chunk` node for chunk ID 12
Embedding created
Creating `:Chunk` node for chunk ID 13
Embedding created
section number is:2



 1 In this section, private road means a private road used by the public for vehicular traffic with permission of the owner or licensee of the road. 
 2 This Act does not apply to the driving or operation of a mechanically propelled invalid's chair that is used only for the purposes for which it was designed. 
 3 Despite subsection (2), a person must not drive or operate on a highway a mechanically propelled invalid's chair of a type or class prescribed by the Lieutenant Governor in Council as a motor vehicle unless the person complies with this Act. 
 4 [Repealed 1997-31-2.] 
 5 The provisions of this Act respecting the registration and licensing of motor vehicles and trailers and the licensing of drivers of motor vehicles do not apply to prohibit  a the temporary driving, propelling, drawing or moving of an implement of husbandry on a highway by or on behalf of a farmer unless the implement is used  i to carry, on a highway, passengers or goods other than farm produce, supplies, sto


 1 Except as otherwise provided under this Act, the owner of a motor vehicle or trailer must, before it is used or operated on a highway,  a register the motor vehicle or trailer with the Insurance Corporation of British Columbia,   b obtain a licence for its operation under this section, and   c obtain for it an owner's certificate under the Insurance (Vehicle) Act.  
 1.1 An off-road vehicle registered with the Insurance Corporation of British Columbia under the Off-Road Vehicle Act is deemed to be sufficiently registered for the purposes of subsection (1) (a) of this section. 
 2 Despite subsection (1), a trailer towed by a tractor licensed under section 8 need not be licensed. 
 3 The owner must apply for  a registration and licence in the form required by the Insurance Corporation of British Columbia, and   b a motor vehicle liability policy.  
 4 The form of motor vehicle liability policy referred to in subsection (3) (b) must be signed by the owner and delivered to the Insuranc


 1 A registration must not be made or a licence issued under this Act in respect of a vehicle except in the name of the owner of the vehicle. 
 2 If the owner of a vehicle is not an individual, a registration or licence must not be made or issued in respect of the vehicle unless the owner is  a a corporation incorporated under the laws of British Columbia,   b an extraprovincial company within the meaning of the Business Corporations Act, or   c an entity prescribed by or comprised in a class prescribed by the Lieutenant Governor in Council.  
Creating `:Chunk` node for chunk ID 0
Embedding created
section number is:5
Creating `:Chunk` node for chunk ID 0
Embedding created
section number is:6


...


 1 In construing and enforcing this Act or the regulations, the act, omission or failure of any officer, agent or person acting for or employed by a carrier, if within the scope of the officer's, agent's or person's employment, is also deemed to be the act, omission or failure of the carrier. 
 2 If an officer, agent or person acting for or employed by a carrier, performs an act or omits or fails to do something and, as a result of the operation of subsection (1), the carrier is deemed to have performed an act or omitted or failed to do something that results in an offence being committed, a ticket, summons or other document served under the Offence Act is deemed to have been served on the carrier if it has been served on the person who is alleged to have performed the act or omitted or failed to do something that resulted in an offence being committed. 
Creating `:Chunk` node for chunk ID 0
Embedding created
section number is:241-248
Creating `:Chunk` node for chunk ID 0
[]
Embedding


 1 If a peace officer has reasonable grounds to believe that a person  a has driven or operated a motor vehicle on a highway while the person is prohibited from driving a motor vehicle under  i this Act, or   ii the Youth Justice Act, the Youth Criminal Justice Act (Canada) or the Criminal Code,    b has driven or operated a motor vehicle on a highway while the person's driver's licence and his or her right to apply for or obtain a driver's licence are suspended under section 89 (1) (b) or (c), 232 or 233,   c has driven or operated a motor vehicle on a highway while  i the person did not hold a subsisting driver's licence issued under this Act and was not exempt under section 34 from holding a driver's licence issued under this Act, and   ii a notice under section 252 was in place on the driving record of the person,    d has committed an offence under section 148,   e has driven or operated a motor vehicle on a highway in a race or in a stunt and the peace officer intends to charge 


 1 A peace officer who impounds a motor vehicle under section 215.46 or 251 (1) must  a complete a notice of impoundment,   b give a copy of the notice of impoundment to the driver and the person who has custody of the motor vehicle, and   c forward to the superintendent  i a report, in the form established by the superintendent, and   ii a copy of the notice of impoundment.   
 2 When the superintendent receives a copy of the notice of impoundment under subsection (1) (c) (ii), the superintendent must promptly send a copy of that notice by mail to the last known address of the registered owner of the motor vehicle as shown in the records maintained by the Insurance Corporation of British Columbia. 
 2.1 Subsection (2) does not apply if  a the driver was the registered owner of the motor vehicle, and   b there is only one registered owner of the motor vehicle.  
 3 The notice of impoundment must contain  a a statement of the right to have the impoundment reviewed by the superintendent


 1 A motor vehicle impounded under section 215.46 or 251 (1) is to be stored at a place directed by the peace officer. 
 2 The following constitute a lien on the motor vehicle:  a fees, costs, charges and surcharges prescribed by the superintendent in respect of the transportation, towing, care, storage, disposition and other related activities respecting the impoundment of the motor vehicle;   b charges for searches and registrations under the Personal Property Security Act that are reasonably necessary to fulfill the obligations of the person who has custody of the motor vehicle under the impoundment;   c the amount prescribed for a deposit, whether the deposit is paid in advance or not, for a device used to immobilize a motor vehicle, if there is damage to the immobilization device at the end of the period of impoundment.  
 3 The lien described in subsection (2) may be enforced under the Warehouse Lien Act or under subsection (7) of this section. 
 4 The person who has custody of 


 1 Subject to subsection (2), the Lieutenant Governor in Council may make regulations establishing pilot projects to research, test and evaluate matters relating to the Motor Vehicle Act, including, without limitation, the following:  a respecting the use and operation of a vehicle or device to transport people or things;   b respecting the registration, licensing and insurance requirements of a vehicle or device;   c respecting the geographic location of a pilot project;   d respecting public review and comment related to a pilot project;   e respecting the monitoring and evaluation of a pilot project;   f respecting fees in respect of a pilot project;   g respecting enforcement.  
 2 Before making a regulation under this section, the Lieutenant Governor in Council must be satisfied that the proposed pilot project  a is in the public interest,   b will be consistent with the objectives set by government, including objectives related to transportation and road safety, and   c has muni

In [ ]:
for index, Acts in enumerate(Acts_documents[300:301]):
    soup = BeautifulSoup(Acts.get_text(), 'xml')
    extract_data(soup, index,  False)
    #sections = soup.find_all("div", class_='section')
    #if (index > 0):
    #    break

In [ ]:
    kg.query("""
         CREATE VECTOR INDEX `Acts_Updatedchunks` IF NOT EXISTS
          FOR (c:UpdatedChunk) ON (c.textEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 384,
            `vector.similarity_function`: 'cosine'    
         }}
""")

In [ ]:
find_law = """
MATCH (n:UpdatedChunk) WHERE
    n.ActId = $ActId
    WITH n LIMIT 1
    RETURN n
    """

# This cell will create the embeddings for all the UpdatedChunk Nodes

In [ ]:
for index, Acts in enumerate(Acts_documents[400:401]):
    print(index)
    soup = BeautifulSoup(Acts.get_text(), 'xml')
    extract_data(soup, index,  True)

In [ ]:
no_title = 0
for index, Acts in enumerate(Acts_documents[10:]):
    soup = BeautifulSoup(Acts.get_text(), 'xml')
    title = soup.find("act:title")
    if (title):
        title = title.get_text()
        result = kg.query(find_law, 
            params={
                'ActId': title
            })
        if result:
            #print(title + " does not exist")
            extract_data(soup, index,  True)
            break
    else:
        no_title +=1
        #print(soup.get_text())
    #break
#print("No title cound " + no_title)

In [ ]:
#now lets match the chunks and create the links withing the document
search_found = []
for index, Acts in enumerate(Acts_documents):
    soup = BeautifulSoup(Acts.get_text(), 'xml')
    search_found.append(extract_data(soup, index,  True))
    #sections = soup.find_all("div", class_='section')
    if (index >  5):
        break

In [ ]:
print(search_found)

In [ ]:
text= search_found[0][0][0]['chunk']['text']
print(text)
references = re.findall(r"(?i)(?:section)^\s*\d+(?:\s*,\s*\d+)*\s*$", text)
print(references)

In [ ]:
text = "Section 5,6 or 7"
references = re.findall(r"(?i)(?:section)\s+(\d,\d+|\(\d+\))(?:\s*\([a-z]\))?", text)
print(references)

In [ ]:
len(references)

# Vector search

In [34]:
def neo4j_vector_search(question):
  """Search for similar nodes using the Neo4j vector index"""
  query_embedding = embeddings.embed_query(question)  
  vector_search_query = """
    CALL db.index.vector.queryNodes($index_name, $top_k, $question) yield node, score
    RETURN score, node.ActId, node.sectionId, node.sectionName, node.text AS text
  """
  similar = kg.query(vector_search_query, 
                     params={
                      'question': query_embedding, 
                      'index_name':'Acts_Updatedchunks', 
                      'top_k': 10})
  return similar

In [61]:
query = 'When an employee is fired what needs to be done next?' 
vector = embeddings.embed_query(query)
#print(vector)
search_results = neo4j_vector_search(query)

Failed to write data to connection IPv4Address(('neo4j', 7687)) (ResolvedIPv4Address(('172.18.0.3', 7687)))


In [36]:
for index, result in enumerate(search_results):
    print(result['score'])
    #print(result)
    print(result['node.ActId'])
    print(result['node.sectionName'])
    print(result['node.sectionId'])
    print(result['text'])
    print("\n\n")

0.745730996131897
Employment Standards Act
Group terminations
64
1 if the employment of 50 or more employees at a single location is to be terminated within any 2 month period, the employer must give written notice of group termination to all of the following : a each employee who will be affected ; b a trade union certified to represent, or recognized by the employer as the bargaining agent of, any affected employees ; c the minister. 2 the notice of group termination must specify all of the following : a the number of employees who will be affected ; b the effective date or dates of the termination ; c the reasons for the termination. 3 the notice of group termination must be given as follows : a at least 8 weeks before the effective date of the first termination, if 50 to 100 employees will be affected ; b at least 12 weeks before the effective date of the first termination, if 101 to 300 employees will be affected ; c at least 16 weeks before the effective date of the first termina